<a href="https://colab.research.google.com/github/AliKarimiENT/NLP_HW1/blob/main/NLPHW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reading txt file

In [7]:
with open('/content/ShamsDaftar6.doc_filename=ShamsDaftar6.txt') as f:
  lines = f.readlines()
print(lines[0])
print(lines[1])
print(lines[2])
print(lines[5])

﻿دیوان شمس تبریزی (غزلیات)

2501 - 3000

--------------------------------------------------------

گرم سیم و درم بودی مرا مونس چه کم بودی	وگر یارم فقیرستی ز زر فارغ چه غم بودی

